In [ ]:
!unzip "/content/stackexchange.zip" -d codes

Streaming output truncated to the last 5000 lines.
  inflating: codes/content/tikz_data/code/tikz_531743_531752_59182.tex  
  inflating: codes/content/tikz_data/code/tikz_551067_551436_166265.tex  
  inflating: codes/content/tikz_data/code/tikz_518762_518763_13598.tex  
  inflating: codes/content/tikz_data/code/tikz_14613_127580_257397.tex  
  inflating: codes/content/tikz_data/code/tikz_418644_418646_879377.tex  
  inflating: codes/content/tikz_data/code/tikz_508789_508816_999460.tex  
  inflating: codes/content/tikz_data/code/tikz_497425_497427_581666.tex  
  inflating: codes/content/tikz_data/code/tikz_535788_535833_219395.tex  
  inflating: codes/content/tikz_data/code/tikz_526488_526522_919838.tex  
  inflating: codes/content/tikz_data/code/tikz_523351_523358_168953.tex  
  inflating: codes/content/tikz_data/code/tikz_41608_511365_705624.tex  
  inflating: codes/content/tikz_data/code/tikz_446292_446296_98748.tex  
  inflating: codes/content/tikz_data/code/tikz_555013_555015_19057

In [ ]:
!apt-get update
!apt-get install -y texlive-latex-base texlive-latex-extra texlive-pictures poppler-utils imagemagick
!pip install torch torchvision transformers pillow
!pip install pytesseract
!pip install pdf2image
!pip install python-slugify

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Pa

In [ ]:
!sed -i 's/rights="none" pattern="PDF"/rights="read|write" pattern="PDF"/' /etc/ImageMagick-6/policy.xml

In [ ]:
import os
import glob
import subprocess
import shutil
from PIL import Image
import pytesseract
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import re
from pathlib import Path
from pdf2image import convert_from_path

In [ ]:
import os
import subprocess
import pandas as pd
from tqdm.notebook import tqdm  # for progress bar in Colab

def compile_existing_tikz(csv_path, output_base_dir="compiled_pdfs2", timeout_seconds=10):
    """
    Compiles existing LaTeX files from CSV paths and adds PDF locations to CSV.

    Args:
        csv_path (str): Path to CSV with 'latex_file_path' column containing .tex paths
        output_base_dir (str): Root directory for compiled PDFs
        timeout_seconds (int): Max time allowed for compilation per file
    """
    os.makedirs(output_base_dir, exist_ok=True)
    df = pd.read_csv(csv_path)
    pdf_paths = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Compiling LaTeX"):
        tex_path = row.get('latex_file_path', '')
        pdf_path = ''

        if not os.path.exists(tex_path):
            print(f"File not found: {tex_path}")
            pdf_paths.append(pdf_path)
            continue

        try:
            rel_path = os.path.relpath(os.path.dirname(tex_path), start=os.getcwd())
            output_dir = os.path.join(output_base_dir, rel_path)
            os.makedirs(output_dir, exist_ok=True)

            compile_command = [
                "pdflatex",
                "-output-directory", output_dir,
                "-interaction=nonstopmode",
                "-halt-on-error",
                tex_path
            ]

            result = subprocess.run(
                compile_command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                timeout=timeout_seconds  # <-- skip if takes too long
            )

            if result.returncode == 0:
                base_name = os.path.splitext(os.path.basename(tex_path))[0]
                pdf_path = os.path.join(output_dir, f"{base_name}.pdf")

                for ext in [".aux", ".log", ".out"]:
                    aux_file = os.path.join(output_dir, f"{base_name}{ext}")
                    if os.path.exists(aux_file):
                        os.remove(aux_file)

        except subprocess.TimeoutExpired:
            print(f"Timeout: Skipping {tex_path}")
        except Exception as e:
            print(f"Error processing {tex_path}: {str(e)}")

        pdf_paths.append(pdf_path)

    df['pdf_path'] = pdf_paths
    df.to_csv(csv_path, index=False)
    print(f"Updated CSV with PDF paths. Compiled files in: {output_base_dir}")

# Usage
os.chdir("/content")
compile_existing_tikz("tikz_data/tex_stackexchange_tikz.csv")

Compiling LaTeX:   0%|          | 0/6612 [00:00<?, ?it/s]

Timeout: Skipping tikz_data/code/tikz_557526_557579_515662.tex
Timeout: Skipping tikz_data/code/tikz_549703_549731_755136.tex
Timeout: Skipping tikz_data/code/tikz_549703_549731_310427.tex
Timeout: Skipping tikz_data/code/tikz_548644_548646_309616.tex
Timeout: Skipping tikz_data/code/tikz_548170_548187_122638.tex
Timeout: Skipping tikz_data/code/tikz_548170_548187_983709.tex
Timeout: Skipping tikz_data/code/tikz_547044_547065_976831.tex
Timeout: Skipping tikz_data/code/tikz_139733_139808_178977.tex
Timeout: Skipping tikz_data/code/tikz_544105_544133_123471.tex
Timeout: Skipping tikz_data/code/tikz_542616_542637_665366.tex
Timeout: Skipping tikz_data/code/tikz_543438_543479_996155.tex
Timeout: Skipping tikz_data/code/tikz_543609_543611_29291.tex
Timeout: Skipping tikz_data/code/tikz_532595_532607_103133.tex
Timeout: Skipping tikz_data/code/tikz_532595_532607_180074.tex
Timeout: Skipping tikz_data/code/tikz_541142_541156_380171.tex
Timeout: Skipping tikz_data/code/tikz_537700_538984_3713

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/tikz_data/tex_stackexchange_tikz2.csv')

# Remove rows where 'pdf_path' is empty
df_cleaned = df[df['pdf_path'].notnull()]

# Alternatively, if 'pdf_path' contains NaN values instead of empty strings:
# df_cleaned = df.dropna(subset=['pdf_path'])

# Save the cleaned DataFrame back to a new CSV file
df_cleaned.to_csv('/content/tikz_data/tex_stackexchange_tikz2.csv', index=False)

print("Rows with empty 'pdf_path' have been removed.")

Rows with empty 'pdf_path' have been removed.


In [ ]:
import os
import pandas as pd
from pdf2image import convert_from_path  # Requires: pip install pdf2image poppler-utils

def pdf_to_png_in_csv(csv_path, output_dir, dpi=300):
    """
    Converts PDFs listed in 'pdf_path' column to PNGs and updates CSV with new paths.

    Args:
        csv_path: Path to CSV file with PDF paths
        output_dir: Directory to save PNG images
        dpi: Image resolution (default 300)
    """
    os.makedirs(output_dir, exist_ok=True)
    error_log = []

    try:
        df = pd.read_csv(csv_path, encoding='latin1')
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_path}")
        return

    # Initialize PNG path column
    df['png_path'] = df.get('png_path', '')
    df['png_error'] = df.get('png_error', '')

    for idx, row in df.iterrows():
        pdf_path = row.get('pdf_path', '')
        if not pdf_path or not os.path.exists(pdf_path):
            error_log.append(f"Row {idx}: Invalid PDF path - {pdf_path}")
            df.at[idx, 'png_error'] = f"Invalid PDF path: {pdf_path}"
            continue

        try:
            # Create output subdirectory using PDF filename
            base_name = os.path.splitext(os.path.basename(pdf_path))[0]
            png_output_dir = os.path.join(output_dir, base_name)
            os.makedirs(png_output_dir, exist_ok=True)

            # Convert PDF to PNG
            images = convert_from_path(
                pdf_path,
                dpi=dpi,
                output_folder=png_output_dir,
                fmt='png',
                output_file=base_name,
                single_file=True  # Combine pages into single image
            )

            # Get first page path (if multiple pages)
            png_path = os.path.join(png_output_dir, f"{base_name}.png")

            if images:
                df.at[idx, 'png_path'] = png_path
            else:
                raise Exception("No images converted")

        except Exception as e:
            error_msg = f"Conversion failed: {str(e)}"
            error_log.append(f"Row {idx}: {error_msg}")
            df.at[idx, 'png_error'] = error_msg
            continue

    # Save updated CSV
    df.to_csv(csv_path, index=False)

    # Write error log
    with open(os.path.join(output_dir, "conversion_errors.log"), 'w') as f:
        f.write("\n".join(error_log))

    print(f"Conversion complete with {len(error_log)} errors. See {output_dir}/conversion_errors.log")

In [ ]:
pdf_to_png_in_csv(
    csv_path="tikz_data/tex_stackexchange_tikz2.csv",
    output_dir="tikz_data/pngs",
    dpi=300
)

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (138438756 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Conversion complete with 18 errors. See tikz_data/pngs/conversion_errors.log


In [ ]:
import pandas as pd

def replace_code_column_with_file_content(csv_path, output_csv_path):
    """
    Replaces the 'code' column in a CSV file with content from files specified
    in the 'local_complete_code_path' column.

    Args:
        csv_path (str): Path to the input CSV file.
        output_csv_path (str): Path to save the updated CSV file.
    """
    try:
        # Read the CSV file
        df = pd.read_csv(csv_path, encoding='latin1')

        # Ensure required columns exist
        if 'latex_file_path' not in df.columns:
            print("Error: 'local_complete_code_path' column not found in the CSV.")
            return
        if 'code' not in df.columns:
            df['code'] = ""

        # Replace 'code' column with content from files
        for idx, row in df.iterrows():
            file_path = row.get('latex_file_path', '')
            if not file_path or not os.path.exists(file_path):
                print(f"Row {idx}: Invalid file path - {file_path}")
                df.at[idx, 'code'] = None  # Leave empty if file path is invalid
                continue

            try:
                # Read content from the file
                with open(file_path, 'r', encoding='utf-8') as f:
                    code_content = f.read()
                    df.at[idx, 'code'] = code_content.strip()  # Replace code column
            except Exception as e:
                print(f"Row {idx}: Failed to read file - {e}")
                df.at[idx, 'code'] = None  # Leave empty if error occurs

        # Save updated DataFrame back to a new CSV file
        df.to_csv(output_csv_path, index=False)
        print(f"Updated CSV saved to {output_csv_path}")

    except FileNotFoundError:
        print(f"Error: File not found at {csv_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
replace_code_column_with_file_content(
    csv_path="/content/tikz_data/tex_stackexchange_tikz2.csv",
    output_csv_path="/content/tikz_data/tex_stackexchange_tikz2.csv"
)

Updated CSV saved to /content/tikz_data/tex_stackexchange_tikz2.csv


In [ ]:
import shutil
import os

# Specify the directory path you want to remove
directory_path = "/content/tikz_data/code"

# Check if the directory exists
if os.path.exists(directory_path):
    # Remove the non-empty directory and all its contents
    shutil.rmtree(directory_path)
    print(f"Directory '{directory_path}' has been successfully removed.")
else:
    print(f"Error: Directory '{directory_path}' does not exist.")

Directory '/content/tikz_data/code' has been successfully removed.


In [ ]:
df = pd.read_csv("/content/tikz_data/tikz_data.csv")
df.drop(columns=["latex_file_path", "pdf_path", "png_error"], inplace=True)
df.to_csv("updated_tikz_dataset.csv", index=False)

In [ ]:
os.chdir("/content/codes2")
# Load the CSV file into a DataFrame
df = pd.read_csv('codes2/tikz_data/cleaned_file.csv')

# Remove rows where 'pdf_path' is empty
df_cleaned = df[df['png_path'].notnull()]

# Alternatively, if 'pdf_path' contains NaN values instead of empty strings:
# df_cleaned = df.dropna(subset=['pdf_path'])

# Save the cleaned DataFrame back to a new CSV file
df_cleaned.to_csv('cleaned_file2.csv', index=False)

print("Rows with empty 'png_path' have been removed.")

Rows with empty 'pdf_path' have been removed.


In [ ]:
!zip -r stackexchange3.zip "/content/tikz_data"

updating: content/tikz_data/ (stored 0%)
  adding: content/tikz_data/pngs/ (stored 0%)
  adding: content/tikz_data/pngs/tikz_38605_430787_435359/ (stored 0%)
  adding: content/tikz_data/pngs/tikz_38605_430787_435359/tikz_38605_430787_435359.png (deflated 8%)
  adding: content/tikz_data/pngs/tikz_522561_522568_285423/ (stored 0%)
  adding: content/tikz_data/pngs/tikz_522561_522568_285423/tikz_522561_522568_285423.png (deflated 2%)
  adding: content/tikz_data/pngs/tikz_507701_507703_894222/ (stored 0%)
  adding: content/tikz_data/pngs/tikz_507701_507703_894222/tikz_507701_507703_894222.png (deflated 19%)
  adding: content/tikz_data/pngs/tikz_514093_514134_850040/ (stored 0%)
  adding: content/tikz_data/pngs/tikz_514093_514134_850040/tikz_514093_514134_850040.png (deflated 6%)
  adding: content/tikz_data/pngs/tikz_549470_549474_676781/ (stored 0%)
  adding: content/tikz_data/pngs/tikz_549470_549474_676781/tikz_549470_549474_676781.png (deflated 12%)
  adding: content/tikz_data/pngs/tikz_5

In [ ]:
from google.colab import files
files.download("/content/stackexchange3.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>